In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
mod = "no_audio"
# ── CONFIG ─────────────────────────────────────────────────────────────────────
SR_SIGNAL    = 56      # for padding later, but not needed here
WINDOW_SIZES = [1.0, 2.0, 10.0]
STEP_SIZE    = 0.5
ANN_PER_FRAME_CSV = f'./annotations/per_modality/annotations_{mod}_majority.csv'

# ── Load the per‐frame annotations (this has 'frame'!) ─────────────────────────
df_ann = pd.read_csv(ANN_PER_FRAME_CSV)

def sliding_windows(arr, window_s, step_s, sr, pad=True):
    ws = int(window_s * sr)
    st = int(step_s   * sr)
    for start in range(0, len(arr), st):
        end = start + ws
        seg = arr[start:end]
        if len(seg) < ws:
            if pad:
                seg = np.pad(seg, (0, ws - len(seg)), mode='edge')
            else:
                break
        yield start, seg

# ── For each window size, build fixed‐window CSV ───────────────────────────────
for w in WINDOW_SIZES:
    records = []
    # group by video/segment/participant in the per‐frame DataFrame
    for (vid, seg, pid), sub in df_ann.groupby(['video_id','segment','participant']):
        # now you *can* sort by 'frame'
        sub = sub.sort_values('frame')
        labels = sub['label'].to_numpy()
        ctx   = (vid, seg, pid)

        # slide
        for start_idx, seg_lbls in sliding_windows(labels, w, STEP_SIZE, SR_SIGNAL, pad=True):
            lbl_win = int(seg_lbls.mean() >= 0.5)
            records.append({
                'video_id':            vid,
                'segment':             seg,
                'participant':         pid,
                'window_start_frame':  start_idx,
                'window_size_s':       w,
                'step_s':              STEP_SIZE,
                'label':               lbl_win
            })

    # write out one CSV per window
    out_dir = f'./annotations/fixed/{mod}'
    os.makedirs(out_dir, exist_ok=True)
    out_fname = os.path.join(out_dir, f'annotations_fixed_{int(w)}s.csv')
    pd.DataFrame(records).to_csv(out_fname, index=False)
    print(f"✅ Saved fixed‐window annotations ({w}s) to {out_fname}")


✅ Saved fixed‐window annotations (1.0s) to ./annotations/fixed/no_audio/annotations_fixed_1s.csv
✅ Saved fixed‐window annotations (2.0s) to ./annotations/fixed/no_audio/annotations_fixed_2s.csv
✅ Saved fixed‐window annotations (10.0s) to ./annotations/fixed/no_audio/annotations_fixed_10s.csv


In [2]:
for mod in ["with_audio", "only_audio", "no_audio"]:
    for w in [1, 2, 10]:
        df = pd.read_csv(f"./annotations/fixed/{mod}/annotations_fixed_{w}s.csv")
        print(f"{mod:>12} | {w}s | total: {len(df):>5} | label==1: {(df['label'] == 1).sum():>5}")

  with_audio | 1s | total: 291800 | label==1:  3608
  with_audio | 2s | total: 291800 | label==1:  4767
  with_audio | 10s | total: 291800 | label==1:  6413
  only_audio | 1s | total: 291200 | label==1:  4546
  only_audio | 2s | total: 291200 | label==1:  5992
  only_audio | 10s | total: 291200 | label==1:  8200
    no_audio | 1s | total: 289650 | label==1:  2444
    no_audio | 2s | total: 289650 | label==1:  2005
    no_audio | 10s | total: 289650 | label==1:   613


In [3]:
import os
import pickle
import numpy as np
import pandas as pd
from glob import glob

# ── CONFIG ─────────────────────────────────────────────────────────────────────
PKL_DIR = "./accelerometer/filtered"

mod = "no_audio"
ANN_DIR = f"./annotations/fixed/{mod}"       # contains annotations_fixed_1s.csv, etc.
OUT_DIR = f"features/fixed_really/{mod}"
os.makedirs(OUT_DIR, exist_ok=True)

FS         = 56.0
FILTERED_COLS = ["accelX_filtered", "accelY_filtered", "accelZ_filtered"]

# ── LOAD SIGNALS ────────────────────────────────────────────────────────────────
signals = {}
for p in glob(os.path.join(PKL_DIR, "*.pkl")):
    pid = int(os.path.splitext(os.path.basename(p))[0])
    with open(p, "rb") as f:
        signals[pid] = pickle.load(f)

# ── FEATURE EXTRACTOR ──────────────────────────────────────────────────────────
def extract_fixed_features(df_sig, t0, t1, win_sec):
    """Exactly your old extract_fixed10s_features, parameterized for any win_sec."""
    win_samples = int(win_sec * FS)
    seg = df_sig[(df_sig["time"] >= t0) & (df_sig["time"] < t1)]
    X = seg[FILTERED_COLS].to_numpy()
    L, F = X.shape

    if L == 0:
        X2 = np.zeros((win_samples, F))
    elif L < win_samples:
        pad_rows = np.tile(X[-1], (win_samples - L, 1))
        X2 = np.vstack([X, pad_rows])
    else:
        X2 = X[:win_samples]

    feats = {}
    for i, col in enumerate(FILTERED_COLS):
        arr = X2[:, i]
        feats[f"{col}_mean"]       = arr.mean()
        feats[f"{col}_var"]        = arr.var()
        feats[f"{col}_energy"]     = (arr ** 2).sum()
        d = np.diff(arr)
        feats[f"{col}_deriv_mean"] = d.mean() if len(d) > 0 else 0
        feats[f"{col}_deriv_std"]  = d.std()  if len(d) > 0 else 0

    feats["SMA"] = np.abs(X2).sum()

    if win_samples > 1:
        def safe_corr(x, y):
            return 0.0 if x.std() == 0 or y.std() == 0 else np.corrcoef(x, y)[0, 1]
        x, y, z = X2.T
        feats["corr_xy"] = safe_corr(x, y)
        feats["corr_xz"] = safe_corr(x, z)
        feats["corr_yz"] = safe_corr(y, z)
    else:
        feats.update({k: 0 for k in ("corr_xy","corr_xz","corr_yz")})

    feats["padded_duration_s"]     = win_sec
    feats["real_sample_count"]     = L
    feats["expected_sample_count"] = win_samples
    feats["padding_applied"]       = int(L < win_samples)

    return feats

# ── MAIN LOOP ─────────────────────────────────────────────────────────────────
for win_sec in [1.0, 2.0, 10.0]:
    ann_csv = os.path.join(ANN_DIR, f"annotations_fixed_{int(win_sec)}s.csv")
    df_ann  = pd.read_csv(ann_csv)

    out_records = []
    for _, row in df_ann.iterrows():
        pid = row["participant"]
        df_sig = signals.get(pid)
        if df_sig is None:
            continue

        # compute window bounds from frame + size
        t0 = row["window_start_frame"] / FS
        t1 = t0 + row["window_size_s"]
        win_sec = row["window_size_s"]
        win_samples = int(win_sec * FS)

        # slice & pad/truncate
        feats = extract_fixed_features(df_sig, t0, t1, win_sec)

        # metadata
        feats.update({
            "video_id":    row["video_id"],
            "segment":     row["segment"],
            "participant": pid,
            "window_start_frame": row["window_start_frame"],
            "window_size_s":      win_sec,
            "label":              row["label"]
        })
        out_records.append(feats)

    # write out one CSV per window size
    out_path = os.path.join(OUT_DIR, f"features_{mod}_{int(win_sec)}s.csv")
    pd.DataFrame(out_records).to_csv(out_path, index=False)
    print(f"✅ Saved {len(out_records)} windows → {out_path}")


✅ Saved 278064 windows → features/fixed_really/no_audio/features_no_audio_1s.csv
✅ Saved 278064 windows → features/fixed_really/no_audio/features_no_audio_2s.csv
✅ Saved 278064 windows → features/fixed_really/no_audio/features_no_audio_10s.csv


In [6]:
import pandas as pd
from sklearn.utils import resample

# 1) Load data
df = pd.read_csv('./features/fixed_really/only_audio/features_only_audio_10s.csv')

# 2) Split minority/majority
df_pos = df[df.label == 1]
df_neg = df[df.label == 0]

# 3) Decide target ratio (e.g. 10% positives)
total = len(df)
pos_target = int(total * 0.50)
neg_target = total - pos_target

# 4) Oversample positives up to pos_target
df_pos_res = resample(df_pos,
                      replace=True,
                      n_samples=pos_target,
                      random_state=42)

# 5) Undersample negatives down to neg_target
df_neg_res = resample(df_neg,
                      replace=False,
                      n_samples=neg_target,
                      random_state=42)

# 6) Combine and shuffle
df_bal = pd.concat([df_pos_res, df_neg_res]).sample(frac=1, random_state=42)

# 7) Export
df_bal.to_csv('./features/fixed_really/balanced/only_audio/features_only_audio_10s_balanced.csv', index=False)
print("New positive ratio:", df_bal.label.mean())

New positive ratio: 0.5


In [4]:
# %%
# Cell 1: imports and mixed-train_cross_modality (returns metrics)
import os
import wandb
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score

def train_mixed_cross_modality(
    train_modality: str,
    test_modality: str,
    duration_s: int,
    cfg: dict = None
):
    # Default hyperparameters
    defaults = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    params = defaults.copy()
    if cfg:
        params.update(cfg)

    run = wandb.init(
        project="rf-segment-classification",
        job_type=f"mixed_train_{train_modality}_test_{test_modality}_{duration_s}s",
        config=params
    )
    config = run.config

    # Paths for cont-padded and balanced sliding windows
    base_cont   = "./features/fixed"
    base_slide  = "./features/fixed_really/balanced"

    cont_file   = os.path.join(base_cont, train_modality, f"features_{train_modality}_{duration_s}s.csv")
    slide_file  = os.path.join(base_slide, train_modality, f"features_{train_modality}_{duration_s}s_balanced.csv")
    test_file   = os.path.join(base_slide, test_modality,  f"features_{test_modality}_{duration_s}s_balanced.csv")

    # 1) Load and mix training sets
    df_cont  = pd.read_csv(cont_file)
    df_slide = pd.read_csv(slide_file)
    df_tr    = pd.concat([df_cont, df_slide], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)
    # 2) Load test set
    df_te = pd.read_csv(test_file)

    # 3) Align features
    drop     = ["video_id","segment","participant","start_time","end_time","label",
                "padded_duration_s","original_duration","real_sample_count",
                "expected_sample_count","padding_applied"]
    feats_tr = set(df_tr.columns) - set(drop)
    feats_te = set(df_te.columns) - set(drop)
    common   = sorted(feats_tr & feats_te)

    X_tr, y_tr = df_tr[common].fillna(0), df_tr["label"]
    X_te, y_te = df_te[common].fillna(0), df_te["label"]

    # 4) Train & eval
    clf = RandomForestClassifier(
        n_estimators=config.n_estimators,
        max_depth=config.max_depth,
        class_weight="balanced",
        random_state=config.random_state,
        n_jobs=-1
    )
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)

    report = classification_report(y_te, preds, output_dict=True, zero_division=0)
    f1_mac = f1_score(y_te, preds, average="macro")

    # 5) Log metrics
    metrics = {
        "train_modality": train_modality,
        "test_modality":  test_modality,
        "duration_s":     duration_s,
        "accuracy":       report["accuracy"],
        "f1_macro":       f1_mac,
        "precision_1":    report["1"]["precision"],
        "recall_1":       report["1"]["recall"],
        "f1_1":           report["1"]["f1-score"]
    }
    run.log({k: v for k, v in metrics.items() if k not in ("train_modality","test_modality","duration_s")})

    # 6) Log top 10 importances
    imps = pd.Series(clf.feature_importances_, index=common).nlargest(10)
    for feat, val in imps.items():
        run.log({feat: val})

    run.finish()
    return metrics



In [87]:
# %%
# Cell 1: imports and train_mixed_cross_modality with held-out split from balanced windows
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

def train_mixed_cross_modality(
    train_modality: str,
    duration_s: int,
    holdout_frac: float = 0.2,
    cfg: dict = None
):
    """
    Train on a mix of contiguously-padded segments and balanced sliding windows,
    with a held-out split from the sliding windows as test set.
    """
    # 1) Hyperparameters
    defaults = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    params = defaults.copy()
    if cfg:
        params.update(cfg)

    # 2) File paths (adjust these to your folders)
    base_cont  = "./features/fixed"                      # contiguously-padded features
    base_slide = "./features/fixed_really/balanced"      # balanced sliding-window features
    cont_file  = os.path.join(base_cont, train_modality, f"features_{train_modality}_{duration_s}s.csv")
    slide_file = os.path.join(base_slide, train_modality, f"features_{train_modality}_{duration_s}s_balanced.csv")

    # 3) Load contiguously-padded features
    df_cont = pd.read_csv(cont_file)

    # 4) Load balanced sliding-window features and split into train/test
    df_slide = pd.read_csv(slide_file)
    df_slide_tr, df_slide_te = train_test_split(
        df_slide,
        test_size=holdout_frac,
        stratify=df_slide['label'],
        random_state=params["random_state"]
    )

    # 5) Build mixed training set and test set
    #df_tr = pd.concat([df_cont, df_slide_tr], axis=0)\
              #.sample(frac=1, random_state=params["random_state"])\
              #.reset_index(drop=True)
    df_tr = df_cont
    df_te = df_slide

    # 6) Align features
    drop_cols = [
        "video_id","segment","participant","start_time","end_time","label",
        "padded_duration_s","original_duration","real_sample_count",
        "expected_sample_count","padding_applied"
    ]
    feats_tr = set(df_tr.columns) - set(drop_cols)
    feats_te = set(df_te.columns) - set(drop_cols)
    common   = sorted(feats_tr & feats_te)

    X_tr, y_tr = df_tr[common].fillna(0), df_tr["label"]
    X_te, y_te = df_te[common].fillna(0), df_te["label"]

    # 7) Train & evaluate
    clf = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        class_weight="balanced",
        random_state=params["random_state"],
        n_jobs=-1
    )
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)

    report = classification_report(y_te, preds, output_dict=True, zero_division=0)
    f1_macro = f1_score(y_te, preds, average="macro")

    # 8) Collect metrics
    metrics = {
        "train_modality": train_modality,
        "duration_s":     duration_s,
        "accuracy":       report["accuracy"],
        "f1_macro":       f1_macro,
        "precision_1":    report["1"]["precision"],
        "recall_1":       report["1"]["recall"],
        "f1_1":           report["1"]["f1-score"]
    }
    print(f"Results for {train_modality} @ {duration_s}s:", metrics)
    return metrics

# %%
# Cell 2: iterate over modalities and durations with mixed training and hold-out testing
import pandas as pd

modalities = ["with_audio", "no_audio", "only_audio"]
durations  = [1, 2, 10]

results = []
for mod in modalities:
    for dur in durations:
        res = train_mixed_cross_modality(
            train_modality=mod,
            duration_s=dur,
            holdout_frac=0.2
        )
        results.append(res)

df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv("mixed_fixed_and_sliding_summary.csv", index=False)


Results for with_audio @ 1s: {'train_modality': 'with_audio', 'duration_s': 1, 'accuracy': 0.8938128284212932, 'f1_macro': 0.4853666395787466, 'precision_1': 0.1607981220657277, 'recall_1': 0.014672283307154076, 'f1_1': 0.026890866265375558}
Results for with_audio @ 2s: {'train_modality': 'with_audio', 'duration_s': 2, 'accuracy': 0.8991175462645647, 'f1_macro': 0.47495232448977026, 'precision_1': 0.12874251497005987, 'recall_1': 0.0015350564043981152, 'f1_1': 0.003033937768997389}
Results for with_audio @ 10s: {'train_modality': 'with_audio', 'duration_s': 10, 'accuracy': 0.9002991489604752, 'f1_macro': 0.4774178292346055, 'precision_1': 0.8373983739837398, 'recall_1': 0.0036769955733257174, 'f1_1': 0.007321841123156211}
Results for no_audio @ 1s: {'train_modality': 'no_audio', 'duration_s': 1, 'accuracy': 0.8914422578974625, 'f1_macro': 0.5027267835577748, 'precision_1': 0.2302810516772439, 'recall_1': 0.0365388765014745, 'f1_1': 0.0630703333540257}
Results for no_audio @ 2s: {'train

In [83]:
# %%
# Cell 2: iterate over modalities and durations with mixed training
wandb.login()

modalities = ["with_audio", "no_audio", "only_audio"]
durations  = [1, 2, 10]

results = []
for mod in modalities:
    for dur in durations:
        print(f"▶ Mixed train on {mod}, test on {mod}, window {dur}s")
        res = train_mixed_cross_modality(train_modality=mod, test_modality=mod, duration_s=dur)
        results.append(res)

df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv("mixed_fixed_and_sliding_summary.csv", index=False)


▶ Mixed train on with_audio, test on with_audio, window 1s


SMA,▁
accelX_filtered_deriv_std,▁
accelY_filtered_deriv_std,▁
accelY_filtered_mean,▁
accelY_filtered_var,▁
accelZ_filtered_deriv_std,▁
accuracy,▁
corr_xz,▁
corr_yz,▁
f1_1,▁
f1_macro,▁


▶ Mixed train on with_audio, test on with_audio, window 2s


SMA,▁
accelX_filtered_deriv_std,▁
accelY_filtered_deriv_std,▁
accelZ_filtered_deriv_std,▁
accelZ_filtered_energy,▁
accelZ_filtered_var,▁
accuracy,▁
corr_xz,▁
corr_yz,▁
f1_1,▁
f1_macro,▁


▶ Mixed train on with_audio, test on with_audio, window 10s


...

▶ Mixed train on only_audio, test on only_audio, window 2s


SMA,▁
accelX_filtered_deriv_std,▁
accelX_filtered_mean,▁
accelX_filtered_var,▁
accelY_filtered_deriv_std,▁
accelY_filtered_mean,▁
accelZ_filtered_deriv_std,▁
accelZ_filtered_var,▁
accuracy,▁
f1_1,▁
f1_macro,▁


▶ Mixed train on only_audio, test on only_audio, window 10s


SMA,▁
accelX_filtered_deriv_std,▁
accelY_filtered_deriv_std,▁
accelY_filtered_energy,▁
accelY_filtered_var,▁
accelZ_filtered_deriv_std,▁
accelZ_filtered_energy,▁
accelZ_filtered_var,▁
accuracy,▁
corr_xz,▁
f1_1,▁


  train_modality test_modality  duration_s  accuracy  f1_macro  precision_1  \
0     with_audio    with_audio           1  0.443729  0.408280     0.151827   
1     with_audio    with_audio           2  0.696789  0.595054     0.245202   
2     with_audio    with_audio          10  0.975686  0.938956     0.804411   
3       no_audio      no_audio           1  0.453845  0.415857     0.154136   
4       no_audio      no_audio           2  0.720251  0.607713     0.253356   
5       no_audio      no_audio          10  0.942003  0.869938     0.634933   
6     only_audio    only_audio           1  0.438970  0.404945     0.151199   
7     only_audio    only_audio           2  0.662134  0.568725     0.226345   
8     only_audio    only_audio          10  0.985759  0.962777     0.875344   

   recall_1      f1_1  
0  0.994859  0.263449  
1  0.977831  0.392084  
2  1.000000  0.891605  
3  0.994174  0.266893  
4  0.923218  0.397599  
5  0.988204  0.773124  
6  0.999249  0.262655  
7  0.983724  0.36